In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import json
import time

def login(driver, site_info):
    """
    주어진 사이트 정보에 따라 로그인을 수행합니다.
    """
    try:
        driver.get(site_info["login_url"])
        print(f"로그인 페이지 이동: {site_info['login_url']}")

        # ID 입력
        id_input = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["id_selector"]))
        )
        id_input.send_keys(site_info["id"])

        # PW 입력
        pw_input = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["pw_selector"]))
        )
        pw_input.send_keys(site_info["pw"])

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, site_info["login_button_selector"]))
        )
        login_button.click()

        time.sleep(3)  # 로그인 후 페이지 이동을 위한 대기

        print("로그인 시도 후 현재 URL:", driver.current_url)
        print("로그인 시도 후 쿠키:", driver.get_cookies())

        # 로그인 성공 여부 확인 (주문조회 페이지 버튼이 있는지 체크)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["order_open_selector"]))
        )
        print("✅ 로그인 성공!")
        print("로그인 성공 후 쿠키:", driver.get_cookies())

        return True

    except Exception as e:
        print(f"❌ 로그인 중 오류 발생: {e}")
        return False


def navigate_to_order_catalog(driver, site_info):
    """
    주문배송조회 페이지로 이동하는 함수 (여러 방법 시도)
    """
    try:
        print("------------------")
        print("주문배송조회 페이지 이동 시도 전 쿠키:", driver.get_cookies())

        # 1️⃣ [기본 이동] driver.get() 사용
        driver.get("https://www.dometopia.com/mypage/order_catalog")
        time.sleep(3)  # 로딩 대기

        print("✅ 페이지 이동 시도: https://www.dometopia.com/mypage/order_catalog")
        print("이동 후 현재 URL:", driver.current_url)

        # 주문조회 테이블이 로드되는지 확인
        order_catalog_selector = "#order_history_table > tbody > tr:nth-child(1) > td:nth-child(1)"
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, order_catalog_selector))
        )
        print("✅ 주문배송조회 페이지 로딩 확인!")

        return True

    except TimeoutException:
        print("⚠️ 주문배송조회 페이지 로딩 실패 (Timeout)")
    
    except Exception as e:
        print(f"❌ 주문배송조회 페이지 이동 중 오류 발생: {e}")

    # 2️⃣ [JavaScript 이동] execute_script 사용
    print("🛠️ URL 이동이 실패하여 JavaScript로 이동 시도...")
    try:
        driver.execute_script("window.location.href='https://www.dometopia.com/mypage/order_catalog';")
        time.sleep(3)

        print("JS 이동 후 현재 URL:", driver.current_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, order_catalog_selector))
        )
        print("✅ JS 이동 성공!")
        return True

    except Exception as e:
        print(f"❌ JavaScript 이동 실패: {e}")

    # 3️⃣ [버튼 클릭 이동] 직접 클릭 시도
    print("🛠️ 버튼 클릭으로 이동 시도...")
    try:
        order_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "주문배송조회"))
        )
        order_button.click()
        time.sleep(3)

        print("클릭 후 현재 URL:", driver.current_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, order_catalog_selector))
        )
        print("✅ 버튼 클릭 이동 성공!")
        return True

    except Exception as e:
        print(f"❌ 버튼 클릭 이동 실패: {e}")

    return False


if __name__ == "__main__":
    # JSON 파일 로드
    json_file_path = r"C:\Users\jh\택배송장\site_configs\dmtopia_config.json"
    with open(json_file_path, "r", encoding="utf-8") as f:
        site_info = json.load(f)

    # Chrome 옵션 설정 (쿠키 유지를 위해 user-data-dir 옵션 추가)
    options = webdriver.ChromeOptions()
    options.add_argument("--user-data-dir=C:\\selenium\\chrome-data")  # **절대 경로로 수정!**
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Whale/4.30.291.11 Safari/537.36")

    # WebDriver 설정 (Chrome 옵션 적용)
    driver = webdriver.Chrome(options=options)

    # 로그인 시도
    if login(driver, site_info):
        print("------------------")
        
        # 주문배송조회 페이지로 이동 (여러 방법 시도)
        if navigate_to_order_catalog(driver, site_info):
            print("------------------")
            print("✅ 주문배송조회 페이지 이동 성공!")
            print("현재 URL:", driver.current_url)
        else:
            print("------------------")
            print("❌ 주문배송조회 페이지 이동 실패.")

    else:
        print("------------------")
        print("❌ 로그인 실패.")

    # WebDriver 종료
    driver.quit()


로그인 페이지 이동: https://www.dometopia.com/member/login
로그인 시도 후 현재 URL: https://www.dometopia.com/main/index
로그인 시도 후 쿠키: [{'domain': '.dometopia.com', 'expiry': 1774669082, 'httpOnly': False, 'name': '_ga_PXB0TQ9W0V', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1740106751.1.1.1740109082.32.0.0'}, {'domain': 'www.dometopia.com', 'expiry': 1774669082, 'httpOnly': False, 'name': 'wcs_bt', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 's_ead6d4ef0f3:1740109082'}, {'domain': '.dometopia.com', 'httpOnly': False, 'name': 'PHPSESSID', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'd3f212682ffb641eda6f7ba32faf7a82'}, {'domain': 'www.dometopia.com', 'expiry': 1771645077, 'httpOnly': False, 'name': 'mobileapp', 'path': '/main', 'sameSite': 'Lax', 'secure': False, 'value': 'N'}, {'domain': '.www.dometopia.com', 'expiry': 1774668602, 'httpOnly': False, 'name': 'wcs_bt', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 's_ead6d4ef0f3:1740108602'}, {'